In [1]:
import requests
import json
from lxml import etree
import pandas as pd
import re
import math

##调用getMerchantComment接口获取评论json
def getShopPage(shopID,offSet):
    simulateChromeBrowserData = {
        'Accept':'*/*',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        'Host':'sz.meituan.com',
        'Referer':'http://wh.meituan.com/meishi/',
        # 'Cookie': '_lxsdk_cuid=16212a00d8ec8-07cdb6596bad8e-178123e-1fa400-16212a00d8fc8; lat=22.780886; lng=113.906362; client-id=34908f62-ea11-4211-b60a-f62c32288b2e; uuid=9be4f96971ac4c9cab4c.1520730903.1.0.0; webloc_geo=22.527181%2C113.938582%2Cwgs84; ci=30; _lxsdk=16212a1cddfc8-011369480302e7-178123e-1fa400-16212a1cddfc8; __mta=247430459.1520730902128.1520731016684.1520731025187.5; _lxsdk_s=16212a00d8f-c83-6e-376%7C%7C9',
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'
    }
    purl='https://www.meituan.com/meishi/api/poi/getMerchantComment'
    params = {
        'id' : shopID,
        'userId' : 0,
        'offset' : offSet*10,#偏移量，也是页码信息
        'pageSize' : 10,
        'sortType' : 1
    }
    page = requests.get(purl,params=params,headers=simulateChromeBrowserData)
    return page



In [2]:
def dataClean(comment):  #只保留中文
    pattern = re.compile(r'[^\u4e00-\u9fa5]')
    commentc = re.sub(pattern, '', comment)
    return commentc

def parse_POI_Shop_Json(shop_json):
    one_comment_data = pd.DataFrame()
    if 'data'in shop_json:
        for eachShopInfo in shop_json.get('data').get('comments'):
            Comment = dataClean(eachShopInfo.get('comment'))
            if not Comment:
                continue
            IsGreet = 1 if eachShopInfo.get('star')>= 30 else 0
            one_comment_data = one_comment_data.append(pd.DataFrame({'Comment':[Comment],'IsGreet':[IsGreet]}))
    return one_comment_data

In [3]:

Comment_datas = pd.DataFrame();
# allCommentNum商家的最大评论数
allCommentNum=9000
for i in range(allCommentNum//10):
    page = getShopPage(74779,i)
    #print(page.url)
    # print(page.json())
    Comment_data = parse_POI_Shop_Json(page.json())
    Comment_datas = Comment_datas.append(Comment_data)
print(Comment_datas)
Comment_datas.to_csv('Comment_datas.csv', index=False)

                                              Comment  IsGreet
0                                个人吃加了个羊腿不错的一家西餐厅还会光顾        1
0   我和男朋友是秀玉的老顾客了第一次认识就是在秀玉吃的饭他们家的环境优雅菜品味道也挺不错今天和男...        1
0   每次来秀玉觉得心情很好因为环境不错东西又很好吃因为没有拍照的习惯只能拿朋友做的东西来凑了貌似...        1
0   和室友一起来的这里位置很好找整个装饰也很有格调适合人们休息点了香辣咖喱鸡焗饭冰淇淋水果圣代奶...        1
0                                  还不错吧环境优雅菜品种类多味道还可以        1
0                               不好吃都浪费了真不知道秀玉现在怎么这么差了        0
0                                     味道还不错环境比较好就餐很安静        1
0                          环境特别好牛排也特别美味金桔茶特别推荐哦就餐特别推荐        1
0                                        还可以人不多上菜快服务好        1
0               带着弟弟们来吃的位置有点难得找人很多味道不错看来地方不同秀玉的味道差别也大        1
0   邓平小姐姐服务态度很好因为是第一次来这里耐心给我们推荐菜品回答我们的问题这里二楼比较安静喜欢...        1
0                                    味道好吃可以可以男朋友吃的很满意        1
0   环境还不错主要是吃这个衣服上不会有味道饮料也很好喝不是那种用粉冲出来的服务员态度也都很好给我...        1
0                                      味道不错套餐实惠小孩很喜欢吃        1
0                                           味道好很喜欢吃推荐  

In [4]:
import numpy as np
X_data=np.array(Comment_datas.get('Comment'))
Y_data=np.array(Comment_datas.get('IsGreet'))
X_test_data=X_data[1400:1485]
Y_test_data=Y_data[1400:1485]
X_train = []
y_train = []
X_test = []
y_test = []
nums = np.arange(1400)
# 随机打乱12000个训练数据
np.random.shuffle(nums)
for i in nums:
    X_train.append(X_data[1])
    y_train.append(Y_data[1])

# 随机打乱1600个测试数据
nums_ = np.arange(85)
np.random.shuffle(nums_)  # shuffle改变的是自身的内容
for i in nums_:
    X_test.append(X_test_data [1])
    y_test.append(Y_test_data [1])

# list转Numpy数组
X_train= np.array(X_train)
X_test= np.array(X_test)

In [9]:
import pkuseg
seg = pkuseg.pkuseg()           # 以默认配置加载模型
for items in X_data:
    text = seg.cut(items)  # 进行分词  
    print(text)


['个人', '吃', '加', '了', '个', '羊腿', '不错', '的', '一家', '西餐厅', '还', '会', '光顾']
['我', '和', '男朋友', '是', '秀玉', '的', '老', '顾客', '了', '第一', '次', '认识', '就是', '在', '秀玉', '吃', '的', '饭', '他们', '家', '的', '环境', '优雅', '菜品', '味道', '也', '挺', '不错', '今天', '和', '男朋友', '再次', '来到', '秀玉', '服务员', '王小凤', '接待', '的', '我们', '服务', '态度', '非常', '好', '值得', '推荐', '小吃', '鸡翅', '经典', '西冷', '甜点', '金桔', '柠檬', '茶红', '烧', '牛蛙', '煲仔']
['每次', '来', '秀玉', '觉得', '心情', '很', '好', '因为', '环境', '不错', '东西', '又', '很', '好吃', '因为', '没有', '拍照', '的', '习惯', '只能', '拿', '朋友', '做', '的', '东西', '来', '凑', '了', '貌似', '我', '太', '懒', '了', '最', '喜欢', '和', '朋友', '来', '一起', '吃', '个', '饭', '聊天', '很', '惬意', '很', '放松', '下次', '来', '试试', '下午', '茶', '应该', '也', '会', '不错', '的']
['和', '室友', '一起', '来', '的', '这里', '位置', '很', '好找', '整个', '装饰', '也', '很', '有', '格调', '适合', '人们', '休息', '点', '了', '香辣', '咖喱', '鸡焗饭', '冰淇淋', '水果', '圣代', '奶油', '培根', '意面', '牛肉', '汉堡', '水果', '沙拉焗饭', '味道', '不错', '芝士', '咖喱', '份量', '足', '水果', '圣代选', '的', '是', '树莓', '和', '香草味', '的', '冰淇淋', '这', '两',

['还', '行', '不错', '味道', '还', '行', '可以', '的']
['点', '的', '三', '样', '菜品', '全部', '踩雷', '番茄', '牛腩', '跟', '饭', '上来', '是', '凉', '的', '而且', '酱汁', '很', '难吃']
['主要', '是', '环境', '好', '习惯', '了', '在', '秀玉', '里', '吃饭']
['环境', '不错', '左右', '的', '牛排', '做', '的', '不', '怎么样']
['汉堡', '划算', '又', '好吃', '环境', '特别', '高大上', '新', '会员', '还有', '送', '小食']
['口味', '不错', '服务', '很', '好']
['味道', '一般', '环境', '好', '服务员', '态度', '好']
['环境', '很', '好', '曹燕', '的', '服务', '态度', '很', '好', '以后', '会', '经常', '光顾']
['头', '一', '次', '在', '光谷', '秀', '玉红', '茶坊', '吃饭', '环境', '非常', '简约', '大气', '很', '有', '宜家', '的', '风格', '里面', '有', '一个', '服务员', '王', '小', '凤姐', '姐人', '非常', '友好', '服务', '很', '贴心', '在', '这里', '给', '五星', '好评', '菜品', '价格', '性价比', '非常', '高', '尤其是', '披萨', '简直', '良心', '价', '了', '金桔', '柠檬', '茶', '既', '美观', '又', '好', '喝', '强烈', '推荐', '总之', '是', '很', '的', '一', '次', '用餐', '个', '赞']
['菜品', '都', '很', '好吃', '蜂蜜厚', '多士', '堪称', '经典', '性价比', '高', '难得', '的', '好', '店']
['每次', '到', '光谷', '来', '都', '是', '在', '他们', '家', '吃饭', '菜品', '份量', '充足', '上'

['菜', '都', '上', '了', '餐具', '没', '到', '菜', '吃完', '了', '饭', '还', '没', '来', '等', '了', '一个', '多', '小时', '催', '了', '好几', '次', '了', '最后', '告诉', '我', '单子', '搞漏', '了', '加急', '做', '了', '个', '煲仔饭', '搞', '成', '了', '锅巴饭', '还', '糊', '了', '一大', '片']
['牛肋', '排', '做', '的', '全', '熟', '烤焦', '了', '全国', '应该', '就此', '一家', '服务员', '一本正经', '的', '说', '只能', '做', '全', '熟']
['上', '菜快', '跟', '汉口', '这边', '的', '菜单', '不', '一样', '没有', '吃到', '我', '想', '吃', '的']
['味道', '不错', '分量', '也', '够', '三', '个人', '吃', '撑', '了']
['服务', '不行', '口味', '可以', '等位', '不用', '等', '环境', '可以']
['环境', '现在', '不咋的', '虫子', '飞来飞去', '的']
['还', '可以', '味道', '不错', '还', '需', '改进']
['除了', '上餐', '的', '服务', '较', '好', '整体', '服务员', '有待', '培训', '提高', '素养', '吃', '了', '这么', '多年', '秀玉', '唯一', '一', '次', '不', '好的', '体验']
['非常', '适合', '约会', '的', '地方', '环境', '很', '好', '也', '很', '好吃', '和', '我', '好', '朋友', '还有', '她', '男朋友', '一起', '来', '的', '这个', '地方', '很', '给面子', '分量', '好多', '我', '还', '点', '了', '份', '牛排', '我们', '三', '个人', '都', '没有', '吃完', '下次', '还', '会', '来', '的', '给',

['第一', '次', '去', '那个', '店', '二楼', '高', '逼格']
['环境', '挺', '好的', '服务', '态度', '很', '满意']
['东西', '还', '行', '吧', '味道', '也', '就', '那个', '样子', '吧']
['感觉', '一般', '吧', '牛排', '不是', '很', '好吃']
['味道', '不行', '服务', '态度', '差', '上', '菜', '速度', '慢']
['很', '好', '朋友', '聚餐', '都', '很', '喜欢']
['想', '吃', '牛排', '推荐', '的', '牛排', '真心', '不', '好吃', '那个', '慢烤', '的', '其他', '的', '还', '不错']
['番茄', '牛腩', '跟', '饭', '那', '米饭硬', '的', '像', '子弹', '还有', '番茄', '牛腩', '太', '酸', '了']
['味道', '还', '行肉酱', '意粉', '有点', '腻人', '可能', '最近', '都', '不', '会', '在', '想', '吃', '了', '分量', '还是', '可以', '的', '上', '菜', '很快']
['味道', '不', '好', '环境', '不错', '只能', '说', '环境', '不错']
['挺', '好吃', '的', '也', '挺', '实惠', '的']
['就', '这样', '吧']
['海鲜', '炒饭', '最', '好吃', '服务', '也', '很', '好', '主要', '是', '干净', '卫生']
['跟', '老公', '一起', '去', '的', '还', '可以', '味道', '还', '行', '吧', '地段', '蛮', '好']
['老', '顾客', '了', '应该', '不', '知道', '吃', '什么', '这个', '一直', '是', '首选']
['还', '不错', '两', '个人', '吃', '很', '饱', '环境', '也', '不错']
['服务', '很', '好']
['一般', '吧', '胜', '在', '量', '多', '吃', '多'

['好久', '没', '来', '了', '没', '以前', '的', '秀玉', '好', '了', '菜品', '份量', '价格', '都', '不如', '以前']
['喜欢', '秀玉', '环境', '非常', '不错', '适合', '谈', '事情']
['环境', '好', '最', '主要', '挺', '好的', '服务', '非常', '到位', '菜品', '质量', '也', '不错', '下次', '还', '会', '再', '来']
['环境', '不错', '约会', '和', '朋友', '聚会', '的', '好', '去处']
['味道', '很', '好', '上', '菜', '速度', '也', '很快', '服务员', '态度', '也', '很', '亲和']
['很', '不错', '以前', '的', '套餐', '就是', '划算', '呀', '现在', '涨', '到', '多', '了']
['我', '就', '喜欢', '那里', '的', '环境', '觉得', '很', '舒服']
['味道', '棒棒', '哒肉', '很', '嫩']
['半年', '没', '评价', '过', '了', '牛排', '很', '嫩', '鸡蛋', '很', '嫩金', '桔汁', '也', '跟', '正宗', '吃', '的', '很', '饱', '而且', '上', '菜', '速度', '特别', '快', '服务', '态度', '也', '挺', '好的', '在', '上', '完', '菜', '后', '还', '会', '主动', '给', '你', '加水']
['还', '不错', '哦', '已经', '来', '吃', '了', '很多', '次', '了', '真正', '的', '不错']
['牛排', '没有', '以前', '的', '好吃', '肉质', '没有', '以前', '的', '鲜嫩', '服务', '还', '可以', '上菜', '速度', '也', '快', '周末', '去', '人', '比较', '多', '但是', '上菜', '速度', '可以', '呢', '就是', '结账', '打', '发票', '的', '时候', '和', 

...

['好', '还', '想', '再', '团']
['好评']
['不错', '下次', '还', '来']
['服务', '差些', '等', '餐', '时间', '长']
['味道', '好', '环境', '好']
['很', '好', '下次', '再', '来']
['还', '不错', '提', '好的']
['好吃']
['还', '可以', '吧']
['蛮', '好']
['好', '还', '想', '再', '团']
['好评']
['不错', '下次', '还', '来']
['服务', '差些', '等', '餐', '时间', '长']
['味道', '好', '环境', '好']
['很', '好', '下次', '再', '来']
['还', '不错', '提', '好的']
['好吃']
['还', '可以', '吧']
['蛮', '好']
['好', '还', '想', '再', '团']
['好评']
['不错', '下次', '还', '来']
['服务', '差些', '等', '餐', '时间', '长']
['味道', '好', '环境', '好']
['很', '好', '下次', '再', '来']
['还', '不错', '提', '好的']
['好吃']
['还', '可以', '吧']
['蛮', '好']
['好', '还', '想', '再', '团']
['好评']
['不错', '下次', '还', '来']
['服务', '差些', '等', '餐', '时间', '长']
['味道', '好', '环境', '好']
['很', '好', '下次', '再', '来']
['还', '不错', '提', '好的']
['好吃']
['还', '可以', '吧']
['蛮', '好']
['好', '还', '想', '再', '团']
['好评']
['不错', '下次', '还', '来']
['服务', '差些', '等', '餐', '时间', '长']
['味道', '好', '环境', '好']
['很', '好', '下次', '再', '来']
['还', '不错', '提', '好的']
['好吃']
['还', '可以', '吧']
['蛮', '好']
['好', '还',

['服务', '差些', '等', '餐', '时间', '长']
['味道', '好', '环境', '好']
['很', '好', '下次', '再', '来']
['还', '不错', '提', '好的']
['好吃']
['还', '可以', '吧']
['蛮', '好']
['好', '还', '想', '再', '团']
['好评']
['不错', '下次', '还', '来']
['服务', '差些', '等', '餐', '时间', '长']
['味道', '好', '环境', '好']
['很', '好', '下次', '再', '来']
['还', '不错', '提', '好的']
['好吃']
['还', '可以', '吧']
['蛮', '好']
['好', '还', '想', '再', '团']
['好评']
['不错', '下次', '还', '来']
['服务', '差些', '等', '餐', '时间', '长']
['味道', '好', '环境', '好']
['很', '好', '下次', '再', '来']
['还', '不错', '提', '好的']
['好吃']
['还', '可以', '吧']
['蛮', '好']
['好', '还', '想', '再', '团']
['好评']
['不错', '下次', '还', '来']
['服务', '差些', '等', '餐', '时间', '长']
['味道', '好', '环境', '好']
['很', '好', '下次', '再', '来']
['还', '不错', '提', '好的']
['好吃']
['还', '可以', '吧']
['蛮', '好']
['好', '还', '想', '再', '团']
['好评']
['不错', '下次', '还', '来']
['服务', '差些', '等', '餐', '时间', '长']
['味道', '好', '环境', '好']
['很', '好', '下次', '再', '来']
['还', '不错', '提', '好的']
['好吃']
['还', '可以', '吧']
['蛮', '好']
['好', '还', '想', '再', '团']
['好评']
['不错', '下次', '还', '来']
['服务', '差些

['很', '好', '下次', '再', '来']
['还', '不错', '提', '好的']
['好吃']
['还', '可以', '吧']
['蛮', '好']
['好', '还', '想', '再', '团']
['好评']
['不错', '下次', '还', '来']
['服务', '差些', '等', '餐', '时间', '长']
['味道', '好', '环境', '好']
['很', '好', '下次', '再', '来']
['还', '不错', '提', '好的']
['好吃']
['还', '可以', '吧']
['蛮', '好']
['好', '还', '想', '再', '团']
['好评']
['不错', '下次', '还', '来']
['服务', '差些', '等', '餐', '时间', '长']
['味道', '好', '环境', '好']
['很', '好', '下次', '再', '来']
['还', '不错', '提', '好的']
['好吃']
['还', '可以', '吧']
['蛮', '好']
['好', '还', '想', '再', '团']
['好评']
['不错', '下次', '还', '来']
['服务', '差些', '等', '餐', '时间', '长']
['味道', '好', '环境', '好']
['很', '好', '下次', '再', '来']
['还', '不错', '提', '好的']
['好吃']
['还', '可以', '吧']
['蛮', '好']
['好', '还', '想', '再', '团']
['好评']
['不错', '下次', '还', '来']
['服务', '差些', '等', '餐', '时间', '长']
['味道', '好', '环境', '好']
['很', '好', '下次', '再', '来']
['还', '不错', '提', '好的']
['好吃']
['还', '可以', '吧']
['还', '不错', '提', '好的']
['好吃']
['还', '可以', '吧']
['不错', '不错']
['味道', '一般']
['吃', '的', '很', '饱']
['满意']
['不错', '三', '个人', '吃', '是', '绝对',

['还', '行', '吧', '感觉', '没', '一', '期', '店', '好']
['味道', '一般']
['吃', '的', '很', '饱']
['满意']
['不错', '三', '个人', '吃', '是', '绝对', '没', '问题', '的']
['还', '可以', '经常', '去']
['不', '好吃', '品种', '搭配', '的', '感觉', '不', '好']
['吃', '的', '很', '饱']
['满意']
['不错', '三', '个人', '吃', '是', '绝对', '没', '问题', '的']
['还', '可以', '经常', '去']
['不', '好吃', '品种', '搭配', '的', '感觉', '不', '好']
['活动', '力度', '大', '点', '就', '好', '啦']
['蔡', '都', '还', '不错', '上', '的', '也', '挺', '快', '邓平', '服务', '很', '周到']
['环境', '非常', '好', '张玉', '服务', '很', '好', '菜品', '也', '非常', '好吃', '性价比', '超高', '的', '一家', '店', '强烈', '推荐', '哦']
['王小凤', '服务', '态度', '很', '好', '下次', '再', '来']
['环境', '很', '好', '服务', '好']
['谭红', '小', '姐姐', '服务', '好']
['张', '玉', '姐姐', '服务', '的', '很', '好', '菜品', '也', '都', '很', '不错']
['曹娜', '服务', '好']
['唐磊', '小', '哥哥', '服务', '很', '好', '哦']
['曹娜', '服务', '很', '好']
['曹娜', '服务', '好']
['蔡', '都', '还', '不错', '上', '的', '也', '挺', '快', '邓平', '服务', '很', '周到']
['环境', '非常', '好', '张玉', '服务', '很', '好', '菜品', '也', '非常', '好吃', '性价比', '超高', '的', '一家', '店', '强烈

['牛排', '很', '好吃', '环境', '很', '好', '很', '适合', '朋友', '一起', '聊天', '喝茶']
['店员', '曹燕', '服务', '态度', '很', '好', '菜品', '分量', '很', '足', '很', '推荐']
['好吃', '好吃', '给', '服务员', '好评']
['味道', '不错', '曹娜', '服务', '很', '好']
['好', '很', '好', '非常', '好']
['王小凤', '服务', '态度', '很', '好', '下次', '再', '来']
['环境', '很', '好', '服务', '好']
['王小凤', '阿姨', '服务', '态度', '很', '好', '下次', '还', '来']
['牛排', '很', '好吃', '环境', '很', '好', '很', '适合', '朋友', '一起', '聊天', '喝茶']
['店员', '曹燕', '服务', '态度', '很', '好', '菜品', '分量', '很', '足', '很', '推荐']
['好吃', '好吃', '给', '服务员', '好评']
['味道', '不错', '曹娜', '服务', '很', '好']
['好', '很', '好', '非常', '好']
['王小凤', '服务', '态度', '很', '好', '下次', '再', '来']
['环境', '很', '好', '服务', '好']
['王小凤', '阿姨', '服务', '态度', '很', '好', '下次', '还', '来']
['牛排', '很', '好吃', '环境', '很', '好', '很', '适合', '朋友', '一起', '聊天', '喝茶']
['店员', '曹燕', '服务', '态度', '很', '好', '菜品', '分量', '很', '足', '很', '推荐']
['好吃', '好吃', '给', '服务员', '好评']
['味道', '不错', '曹娜', '服务', '很', '好']
['好', '很', '好', '非常', '好']
['王小凤', '服务', '态度', '很', '好', '下次', '再', '来']
['环境', '很', 

In [11]:
from service.client import BertClient
bc = BertClient()
# 将待训练的中文数据转换为（，768）维的句向量
input_train = bc.encode(X_train)

ModuleNotFoundError: No module named 'service'

In [ ]:
input_train.to_csv('input_train.csv', index=False)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
import tensorflow as tf
model = Sequential()
# 搭建模型
model.add(Dense(32, activation='relu', input_shape=(768,)))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.train.AdamOptimizer(),
    metrics=['acc']
)
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=128,
    validation_split=0.2,
    verbose=1
)

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
epochs = range(1, len(history_dict['acc']) + 1)
# 绘图部分
plt.figure()
plt.plot(epochs, history_dict['acc'], 'b', label='acc')
plt.plot(epochs, history_dict['val_acc'], 'bo', label='val_acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
test_loss = model.evaluate(
    X_test,
    y_test,
    batch_size=63,
    verbose=1
)
print(test_loss)